In [6]:
!wget https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

--2025-12-01 19:47:33--  https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg
Resolving habrastorage.org (habrastorage.org)... 2a14:b680:0:56::34, 2a14:b680:0:56::35, 95.47.173.34, ...
Connecting to habrastorage.org (habrastorage.org)|2a14:b680:0:56::34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 398272 (389K) [image/jpeg]
Saving to: ‘yf_dokzqy3vcritme8ggnzqlvwa.jpeg’

yf_dokzqy3vcritme8g 100%[===================>] 388.94K   115KB/s    in 3.4s    

2025-12-01 19:47:37 (115 KB/s) - ‘yf_dokzqy3vcritme8ggnzqlvwa.jpeg’ saved [398272/398272]



In [4]:
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL=f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL=f"{PREFIX}/hair_classifier_v1.onnx"
!wget "{DATA_URL}"
!wget "{MODEL_URL}"

--2025-12-01 19:27:18--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T04%3A03%3A05Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T03%3A02%3A28Z&ske=2025-12-02T04%3A03%3A05Z&sks=b&skv=2018-11-09&sig=cRUFrKeSyA%2BjzeOzvVJCfMFU5wHGka7MP%2F7iuO60mkg%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDY0NzgzOSwibmJmIjoxNzY0NjQ2MDM5LCJ

In [2]:
import onnx

model = onnx.load("hair_classifier_v1.onnx")

In [3]:
import onnx.helper
print(onnx.helper.printable_graph(model.graph))

graph main_graph (
  %input[FLOAT, s77x3x200x200]
) initializers (
  %conv1.weight[FLOAT, 32x3x3x3]
  %conv1.bias[FLOAT, 32]
  %fc1.weight[FLOAT, 64x313632]
  %fc1.bias[FLOAT, 64]
  %fc2.weight[FLOAT, 1x64]
  %fc2.bias[FLOAT, 1]
  %val_4[INT64, 1]
) {
  %val_0 = Shape[end = 1, start = 0](%input)
  %conv2d = Conv[auto_pad = 'NOTSET', dilations = [1, 1], group = 1, pads = [0, 0, 0, 0], strides = [1, 1]](%input, %conv1.weight, %conv1.bias)
  %relu = Relu(%conv2d)
  %max_pool2d = MaxPool[auto_pad = 'NOTSET', ceil_mode = 0, dilations = [1, 1], kernel_shape = [2, 2], pads = [0, 0, 0, 0], storage_order = 0, strides = [2, 2]](%relu)
  %val_5 = Concat[axis = 0](%val_0, %val_4)
  %view = Reshape[allowzero = 1](%max_pool2d, %val_5)
  %linear = Gemm[alpha = 1, beta = 1, transA = 0, transB = 1](%view, %fc1.weight, %fc1.bias)
  %relu_1 = Relu(%linear)
  %output = Gemm[alpha = 1, beta = 1, transA = 0, transB = 1](%relu_1, %fc2.weight, %fc2.bias)
  return %output
}


/var/folders/ds/6hryjbzj5g75d79xz48df3hh0000gn/T/ipykernel_25832/2175673656.py:2: DeprecationWarning: Deprecated since 1.19. Consider using onnx.printer.to_text() instead.
  print(onnx.helper.printable_graph(model.graph))


In [4]:
model.graph.output[0].name

'output'

In [5]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [ ]:
prepare_image(download_image("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"), [200, 200])

In [8]:
import torchvision.transforms as transforms

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [21]:
import numpy as np

test_data = train_transforms(download_image("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"))

In [23]:
import onnxruntime as rt

session = rt.InferenceSession("hair_classifier_v1.onnx")

input_name = session.get_inputs()[0].name
label_name = session.get_outputs()[0].name

input_name, label_name

('input', 'output')

In [ ]:
test_data_np = test_data.numpy()  # (3, 200, 200)
test_data_batch = np.expand_dims(test_data_np, axis=0).astype(np.float32)  # (1, 3, 200, 200)

# run inference
result = session.run([label_name], {input_name: test_data_batch})

print(f"raw output: {result}")
print(f"Prediction: {result[0][0][0]:.4f}")

Raw output: [array([[0.72769725]], dtype=float32)]
Prediction: 0.7277
